In [1]:
import logging
import time

import numpy as np
import matplotlib.pyplot as plt
import os
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress INFO and WARNING logs
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_text

2025-05-15 21:50:07.435929: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747335007.495179    1115 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747335007.510079    1115 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747335007.581437    1115 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747335007.581522    1115 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747335007.581526    1115 computation_placer.cc:177] computation placer alr

In [2]:
# Use TensorFlow Datasets to load the Portuguese-English translation dataset!!!
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en',  with_info=True, as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']

E0000 00:00:1747335023.057326    1115 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1747335023.070343    1115 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
def preview_translation_examples(dataset, batch_size: int = 5, num_batches: int = 1):
    """
    Prints sample translation examples from the dataset.

    Args:
        dataset: A `tf.data.Dataset` yielding (pt, en) translation pairs.
        batch_size (int): Number of examples per batch to preview.
        num_batches (int): Number of batches to preview.
    """
    for batch_num, (pt_batch, en_batch) in enumerate(dataset.batch(batch_size).take(num_batches), start=1):
        print(f'--- Batch {batch_num} ---')
        for pt, en in zip(pt_batch.numpy(), en_batch.numpy()):
            print(f'Portuguese: {pt.decode("utf-8")}')
            print(f'English   : {en.decode("utf-8")}\n')

# Usage
preview_translation_examples(train_examples)

--- Batch 1 ---
Portuguese: e quando melhoramos a procura , tiramos a única vantagem da impressão , que é a serendipidade .
English   : and when you improve searchability , you actually take away the one advantage of print , which is serendipity .

Portuguese: mas e se estes fatores fossem ativos ?
English   : but what if it were active ?

Portuguese: mas eles não tinham a curiosidade de me testar .
English   : but they did n't test for curiosity .

Portuguese: e esta rebeldia consciente é a razão pela qual eu , como agnóstica , posso ainda ter fé .
English   : and this conscious defiance is why i , as an agnostic , can still have faith .

Portuguese: `` `` '' podem usar tudo sobre a mesa no meu corpo . ''
English   : you can use everything on the table on me .



In [4]:
# Set up the tokenizer!!
"""
Tokenization is the process of breaking up text, into "tokens".
Download, extract, and import the saved_model:  
"""
model_name = 'ted_hrlr_translate_pt_en_converter'
tf.keras.utils.get_file(
    f'{model_name}.zip',
    f'https://storage.googleapis.com/download.tensorflow.org/models/{model_name}.zip',
    cache_dir='.', cache_subdir='', extract=True
)

tokenizers = tf.saved_model.load(model_name)

184801/184801 ━━━━━━━━━━━━━━━━━━━━ 0s 2us/step


OSError: SavedModel file does not exist at: ted_hrlr_translate_pt_en_converter/{saved_model.pbtxt|saved_model.pb}